# User Churn Prediction

In this project, we use supervised learning models to identify customers who are likely to stop using service in the future. Furthermore, we will analyze top factors that influence user retention.

# Part 0: Read the data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# taking a look at the top 5 rows
churn_df = pd.read_csv('/content/drive/MyDrive/GEC/project/churn.csv.all')
churn_df.head()

# Part 1: Data Exploration

### Part 1.1: Understand the Raw Dataset

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# find how many samples are there in the data
# find how many variables are there in the data
print ("Num of rows: " + str(churn_df.shape[0]))
print ("Num of columns: " + str(churn_df.shape[1]))

In [ ]:
# take a look at all variable name
churn_df.columns

### Part 1.2: Data cleaning

In [ ]:
# checking the categorical features
churn_df['voice_mail_plan'][0]
churn_df['intl_plan'][0]
churn_df['churned'][0]

In [ ]:
np.sum(churn_df['churned']==' False.')

In [ ]:
# lambda function to remove the heading the trailing white space
# apply(lambda x: function_of_x)
churn_df['voice_mail_plan'] = churn_df['voice_mail_plan'].apply(lambda x: x.strip())
churn_df['intl_plan'] = churn_df['intl_plan'].apply(lambda x: x.strip())
churn_df['churned'] = churn_df['churned'].apply(lambda x: x.strip())

In [ ]:
churn_df['voice_mail_plan'][0]
churn_df['intl_plan'][0]
churn_df['churned'][0]

In [ ]:
# check the number of case in different class
# we always want to do this, because we want to know whether or not the data is balanced
# and this will influence how well the model can get
print(sum(churn_df['churned']=='True.'))
print(sum(churn_df['churned']=='False.'))

### Part 1.3:  Understand the features

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# we can plot the distribution of a variable
# it is bell-shaped, almost normally distributed
sns.distplot(churn_df['total_intl_charge'])

In [ ]:
# make a heatmap of the pairwise-correlation
corr = churn_df[["account_length", "number_vmail_messages", "total_day_minutes",
                    "total_day_calls", "total_day_charge", "total_eve_minutes",
                    "total_eve_calls", "total_eve_charge", "total_night_minutes",
                    "total_night_calls", "total_intl_minutes", "total_intl_calls",
                    "total_intl_charge"]].corr()

sns.heatmap(corr)

In [ ]:
corr

# Part 2: Feature Preprocessing

In [ ]:
# if you want to see only one correlation
from scipy.stats import pearsonr
print(pearsonr(churn_df['total_day_minutes'], churn_df['number_vmail_messages'])[0])

In general(especially in the linear regression case), we want to exclude one of the pair that the correlation is higher than 0.9. For the correlation that is less than 0.9 but relative high, we want to consider the interaction term.

In [ ]:
# we want to get ground truth data
y = np.where(churn_df['churned'] == 'False.',0,1)

# we want to drop the columns that is not a reason to the result
to_drop = ['state','area_code','phone_number','churned']
churn_feat_space = churn_df.drop(to_drop, axis=1)

# there are some yes/no column, we want to convert them into True/False
yes_no_cols = ["intl_plan","voice_mail_plan"]
churn_feat_space[yes_no_cols] = churn_feat_space[yes_no_cols] == 'yes'

X = churn_feat_space

In [ ]:
# see what is the shape of the data
X.shape, y.shape

In [ ]:
num = y.sum() / y.shape[0] * 100
print('There are %s percent of y that has value 1'% num)

#### Practice 1: Can you add catogorical features, e.g. state, into your feature matrix? (one suggestion would be one_hot_encoding)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
churn_df['intl_plan']=churn_df['intl_plan'].astype('category')
churn_df['voice_mail_plan']=churn_df['voice_mail_plan'].astype('category')
churn_df['intl_plan_new']=churn_df['intl_plan'].cat.codes
churn_df['voice_mail_plan_new']=churn_df['voice_mail_plan'].cat.codes
encoder=OneHotEncoder()
enc_data=pd.DataFrame(encoder.fit_transform(churn_df[['Gen_new','Rem_new']]).toarray())
New_df=churn_df.join(enc_data)
print(New_df)

# Part 3: Model Training and Result Evaluation

### Part 3.1: Split dataset

In [ ]:
from sklearn import model_selection

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)

print('training data has %d observation with %d features'% X_train.shape)
print('test data has %d observation with %d features'% X_test.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

### Part 3.2: Model Training and Selection

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.linear_model import LogisticRegression

classifier_logistic = LogisticRegression()

classifier_KNN = KNeighborsClassifier()

classifier_RF = RandomForestClassifier()

In [ ]:
classifier_logistic.fit(X_train, y_train)

In [ ]:
classifier_logistic.predict(X_test)

In [ ]:
classifier_logistic.predict_proba(X_test)

In [ ]:
classifier_logistic.score(X_test, y_test)

In [ ]:
model_names = ['Logistic Regression','KNN','Random Forest']
model_list = [classifier_logistic, classifier_KNN, classifier_RF]
count = 0

for classifier in model_list:
    cv_score = model_selection.cross_val_score(classifier, X_train, y_train, cv=5)
    print(cv_score)
    print('Model accuracy of %s is: %.3f'%(model_names[count],cv_score.mean()))
    count += 1

In [ ]:
cv_score.mean()

#### Practice 2: Can you do prediction with SVM model?

In [ ]:
from sklearn import svm
svc = svm.SVC(kernel="linear").fit(X_train, y_train)
from sklearn.model_selection import cross_val_score
c_score = cross_val_score(svc, X_train,y_train, cv=6)
print(c_score)

### Part 3.3: Use Grid Search to Find Optimal Hyperparameters

In [ ]:
from sklearn.model_selection import GridSearchCV

def print_grid_search_metrics(gs):
    print ("Best score: %0.3f" % gs.best_score_)
    print ("Best parameters set:")
    best_parameters = gs.best_params_
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

#### Part 3.3.1: Find Optimal Hyperparameters - LogisticRegression

In [ ]:
parameters = {
    'penalty':('l1', 'l2'), 
    'C':(1, 5, 10)
}
Grid_LR = GridSearchCV(LogisticRegression(),parameters, cv=5)
Grid_LR.fit(X_train, y_train)

In [ ]:
print_grid_search_metrics(Grid_LR)

In [ ]:
best_LR_model = Grid_LR.best_estimator_

#### Part 3.3.2: Find Optimal Hyperparameters: KNN

In [ ]:
parameters = {
    'n_neighbors':[3,5,7,10] 
}
Grid_KNN = GridSearchCV(KNeighborsClassifier(),parameters, cv=5)
Grid_KNN.fit(X_train, y_train)

In [ ]:
print_grid_search_metrics(Grid_KNN)
best_KNN_model = Grid_KNN.best_estimator_

#### Part 3.3.3: 
#### Practice 3: Find Optimal Hyperparameters: Random Forest(using [40,60,80] as parameter space)

### Part 3.4: Model Evaluation - Confusion Matrix (Precision, Recall, Accuracy)

class of interest as positive

TP: correctly labeled real churn

Precision: tp / (tp + fp);
Total number of true predictive churn divided by the total number of predictive churn;
High Precision means low fp, not many return users were predicted as churn users. 


Recall: tp / (tp + fn)
Predict most postive or churn user correctly. High recall means low fn, not many churn users were predicted as return users.

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def cal_evaluation(classifier, cm):
    tn = cm[0][0]
    fp = cm[0][1]
    fn = cm[1][0]
    tp = cm[1][1]
    accuracy  = (tp + tn) / (tp + fp + fn + tn + 0.0)
    precision = tp / (tp + fp + 0.0)
    recall = tp / (tp + fn + 0.0)
    print (classifier)
    print ("Accuracy is: %0.3f" % accuracy)
    print ("precision is: %0.3f" % precision)
    print ("recall is: %0.3f" % recall)

def draw_confusion_matrices(confusion_matricies):
    class_names = ['Not','Churn']
    for cm in confusion_matrices:
        classifier, cm = cm[0], cm[1]
        cal_evaluation(classifier, cm)
        fig = plt.figure()
        ax = fig.add_subplot(111)
        cax = ax.matshow(cm, interpolation='nearest',cmap=plt.get_cmap('Reds'))
        plt.title('Confusion matrix for %s' % classifier)
        fig.colorbar(cax)
        ax.set_xticklabels([''] + class_names)
        ax.set_yticklabels([''] + class_names)
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.show()

In [ ]:
confusion_matrices = [
    ("Random Forest", confusion_matrix(y_test,best_RF_model.predict(X_test))),
    ("Logistic Regression", confusion_matrix(y_test,best_LR_model.predict(X_test))),
]

draw_confusion_matrices(confusion_matrices)

### Part 3.4: Model Evaluation - ROC & AUC

RandomForestClassifier, KNeighborsClassifier and LogisticRegression have predict_prob() function 

#### Part 3.4.1: ROC of RF Model

In [ ]:
from sklearn.metrics import roc_curve
from sklearn import metrics

y_pred_rf = best_RF_model.predict_proba(X_test)[:, 1]
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_rf)

In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_rf, tpr_rf, label='RF')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve - RF model')
plt.legend(loc='best')
plt.show()

In [ ]:
from sklearn import metrics

metrics.auc(fpr_rf,tpr_rf)

#### Part 3.4.2: ROC of LR Model

In [ ]:
y_pred_lr = best_LR_model.predict_proba(X_test)[:, 1]
fpr_lr, tpr_lr, _ = roc_curve(y_test, y_pred_lr)

In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_lr, tpr_lr, label='LR')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve - LR Model')
plt.legend(loc='best')
plt.show()

In [ ]:
metrics.auc(fpr_lr,tpr_lr)

# Part 4: Feature Selection

### Part 4.1:  Logistic Regression Model - Feature Selection Discussion 

The corelated features that we are interested in: (total_day_minutes, total_day_charge), (total_eve_minutes, total_eve_charge), (total_intl_minutes, total_intl_charge).

In [ ]:
scaler = StandardScaler()
X_l1 = scaler.fit_transform(X)
LRmodel_l1 = LogisticRegression(penalty="l1", C = 0.1, solver='liblinear')
LRmodel_l1.fit(X_l1, y)
LRmodel_l1.coef_[0]
print ("Logistic Regression (L1) Coefficients")
for k,v in sorted(zip(map(lambda x: round(x, 4), LRmodel_l1.coef_[0]), \
                      churn_feat_space.columns), key=lambda k_v:(-abs(k_v[0]),k_v[1])):
    print (v + ": " + str(k))

In [ ]:
scaler = StandardScaler()
X_l2 = scaler.fit_transform(X)
LRmodel_l2 = LogisticRegression(penalty="l2", C = 0.1)
LRmodel_l2.fit(X_l2, y)
LRmodel_l2.coef_[0]
print ("Logistic Regression (L2) Coefficients")
for k,v in sorted(zip(map(lambda x: round(x, 4), LRmodel_l2.coef_[0]), \
                      churn_feat_space.columns), key=lambda k_v:(-abs(k_v[0]),k_v[1])):
    print (v + ": " + str(k))
  

### Part 4.2:  Random Forest Model - Feature Importance Discussion

In [ ]:
forest = RandomForestClassifier()
forest.fit(X, y)

importances = forest.feature_importances_

print("Feature importance ranking by Random Forest Model:")
for k,v in sorted(zip(map(lambda x: round(x, 4), importances), churn_feat_space.columns), reverse=True):
    print (v + ": " + str(k))